In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

from numpy import pi, log, sin, cos, exp
from riptide.libffa import LibFFA

In [2]:
def get_width_trials(b, wmax=0.25, wtsp=1.5):
    """ Generate the list of width trials """
    widths = []
    w = 1
    wmax = max(1, wmax * b)
    while w <= wmax:
        widths.append(w)
        w = max(w + 1, int(w * wtsp))
    return np.asarray(widths)


def runtime_avg(m, b, snr=True, runs=20):
    data = np.zeros(shape=(m, b), dtype=np.float32)
    out = np.zeros_like(data)
    widths = get_width_trials(b)
    snrs = np.zeros(shape=(m, widths.size), dtype=np.float32)
    varnoise = 1.0

    runtimes = []

    for irun in range(runs):
        start = time.time()
        LibFFA.py_transform(data, m, b, out)
        if snr:
            LibFFA.get_snr_2d(out, m, b, widths, widths.size, varnoise, snrs)
        end = time.time()
        runtimes.append(end - start)

    return sum(runtimes) / runs

def runtime_full_search(tobs, b, period_min=1.0, period_max=60.0, runs=1):
    mmax = int(tobs / period_min)
    
    dsfactor = (b+1.0)/b
    current_period = period_min
    runtimes = []
    
    while current_period < period_max:
        m = int(tobs / current_period)
        runtime = runtime_avg(m, b, runs=runs)
        runtimes.append(runtime)
        #print('%12.4fs %8d %12.4fms' % (current_period, m, runtime))
        current_period *= dsfactor
    return np.asarray(runtimes)

In [ ]:
pmin_trials = [0.256, 0.384, 0.512, 0.768, 1.024, 1.536, 2.048]
pmax = 60.0
bins_trials = [128, 192, 256, 384, 512, 768, 1024]
tobs = 3600.0

for pmin in reversed(pmin_trials):
    for bins in bins_trials:
        runtimes = runtime_full_search(tobs, bins, period_min=pmin)
        print('%10.4fs %8d %10.4fs' % (pmin, bins, runtimes.sum()))

In [9]:
pmin_trials = [0.256, 0.384, 0.512, 0.768, 1.024, 1.536, 2.048]
pmax = 60.0
bins_trials = [128, 192, 256, 384, 512, 768, 1024]
tobs = 560.0

for pmin in reversed(pmin_trials):
    for bins in bins_trials:
        runtimes = runtime_full_search(tobs, bins, period_min=pmin)
        print('%10.4fs %8d %10.4fs' % (pmin, bins, runtimes.sum()))

    2.0480s      128     0.0634s
    2.0480s      192     0.0692s
    2.0480s      256     0.1356s
    2.0480s      384     0.2110s
    2.0480s      512     0.3192s
    2.0480s      768     0.6054s
    2.0480s     1024     1.0288s
    1.5360s      128     0.0499s
    1.5360s      192     0.0871s
    1.5360s      256     0.1571s
    1.5360s      384     0.2658s
    1.5360s      512     0.4116s
    1.5360s      768     0.7906s
    1.5360s     1024     1.3138s
    1.0240s      128     0.0688s
    1.0240s      192     0.1525s
    1.0240s      256     0.2147s
    1.0240s      384     0.3827s
    1.0240s      512     0.6074s
    1.0240s      768     1.1812s
    1.0240s     1024     2.0807s
    0.7680s      128     0.0910s
    0.7680s      192     0.1855s
    0.7680s      256     0.2679s
    0.7680s      384     0.5019s
    0.7680s      512     0.7922s
    0.7680s      768     1.5880s
    0.7680s     1024     2.8018s
    0.5120s      128     0.1273s
    0.5120s      192     0.2662s
    0.5120

In [28]:
runtimes.sum()

7.0759655634562177

In [13]:
491.52e-6 * 384

0.18874367999999997

In [14]:
3600.0/560.0

6.428571428571429